# Extract analysis results and perform verifications

This script reads out specified values form the result dict of each structure of a sampling iteration and saves the results in a df.

From the read out strains and stresses the minimum structural safety $\eta$ ("Erfüllungsgrad") are calculated for each structure.

Then the calculated verification results and the corresponding bridge parameter are saved in a csv-file..

In [1]:
#import packages
import pandas as pd
import os

## Batch Evaluation

In [14]:
# Input HERE
idx_s=65
start_id = 0
end_id = 14
step='step_4'

#calculation 
# from verification import calc_eta

from verification import calc_eta
df_res= calc_eta(idx_s, start_id, end_id, step, extract_from ='results', folder_name='CFBData', with_eta_stresses=True, verbalise=False)

There were 0 Structured that resulted in an error during analysis. (Error IDs: [])


In [15]:
#print
df_res

,eta_min_s,x_s,y_s,z_s,Location_s,GP_count_s,eta_min_c,x_c,y_c,z_c,...,y_c_shear,z_c_shear,Location_c_shear,element_count_shear,eta_min_c_stresses,x_c_stresses,y_c_stresses,z_c_stresses,Location_c_stresses,ID
0,7.150362,225.948547,160.000000,-1393.869385,eta_4,0,15.146850,225.948547,160.000000,-1923.530518,...,0.0,-1658.7,bot,0,7.832483,225.948547,160.000000,-1923.530518,eta_y_bot_stresses,0
0,8.431634,225.948547,154.285721,-1393.869385,eta_4,0,15.240659,225.948547,154.285721,-1923.530518,...,0.0,-1658.7,bot,0,7.879340,225.948547,154.285721,-1923.530518,eta_y_bot_stresses,1
0,9.730337,225.948547,152.941177,-1393.869385,eta_4,0,15.314300,225.948547,152.941177,-1923.530518,...,0.0,-1658.7,bot,0,7.916116,225.948547,152.941177,-1923.530518,eta_y_bot_stresses,2
0,10.095302,225.948547,1975.996216,-1129.411743,eta_1,0,15.385144,225.948547,152.941177,-1923.530518,...,0.0,-1658.7,bot,0,7.951498,225.948547,152.941177,-1923.530518,eta_y_bot_stresses,3
0,10.124637,225.948547,1975.996216,-1129.411743,eta_1,0,15.459578,225.948547,152.941177,-1923.530518,...,0.0,-1658.7,bot,0,7.988680,225.948547,152.941177,-1923.530518,eta_y_bot_stresses,4
0,10.151960,225.948547,1975.996216,-1129.411743,eta_1,0,15.533290,225.948547,152.941177,-1923.530518,...,0.0,-1658.7,bot,0,8.025499,225.948547,152.941177,-1923.530518,eta_y_bot_stresses,5
0,10.177699,225.948547,1975.996216,-1129.411743,eta_1,0,15.608333,225.948547,152.941177,-1923.530518,...,0.0,-1658.7,bot,0,8.062981,225.948547,152.941177,-1923.530518,eta_y_bot_stresses,6
0,10.202295,225.948547,1975.996216,-1129.411743,eta_1,0,15.685478,225.948547,152.941177,-1923.530518,...,0.0,-1658.7,bot,0,8.101516,225.948547,152.941177,-1923.530518,eta_y_bot_stresses,7
0,10.225700,225.948547,1975.996216,-1129.411743,eta_1,0,15.762870,225.948547,152.941177,-1923.530518,...,0.0,-1658.7,bot,0,8.140179,225.948547,152.941177,-1923.530518,eta_y_bot_stresses,8
0,10.248333,225.948547,1975.996216,-1129.411743,eta_1,0,15.840386,225.948547,152.941177,-1923.530518,...,0.0,-1658.7,bot,0,8.178907,225.948547,152.941177,-1923.530518,eta_y_bot_stresses,9


In [16]:
#saving calclulated etas in csv-file
folder_name='CFBData'
current_directory = os.getcwd()
folder_path = os.path.join(current_directory, folder_name)
filepath=folder_path+'\\{}_Batch\\{}_CFB_evaluations.csv'.format(idx_s,idx_s)

df_res.to_csv(filepath,index=True)

In [17]:
#merge x and y df and save them too
current_directory = os.getcwd()

y_file_name='CFBData\\{}_Batch\\{}_CFB_evaluations.csv'.format(idx_s,idx_s)
y_file_path = os.path.join(current_directory, y_file_name)
df_y=pd.read_csv(y_file_path).drop(columns=['Unnamed: 0'])

x_file_name='CFBData\\{}_Batch\\{}_CFBSamples.csv'.format(idx_s,idx_s)
x_file_path = os.path.join(current_directory, x_file_name)
df_x=pd.read_csv(x_file_path).rename(columns={'Unnamed: 0':'ID'})

df_xy = pd.merge(df_x, df_y, on='ID', how='inner')

filepath=folder_path+'\\{}_Batch\\{}_CFB_xy.csv'.format(idx_s,idx_s)
df_xy.to_csv(filepath,index=True)

df_xy

,ID,d4_walls,L,b1,t_p,t_w,h_w,alpha_l,h_v,l_v,...,x_c_shear,y_c_shear,z_c_shear,Location_c_shear,element_count_shear,eta_min_c_stresses,x_c_stresses,y_c_stresses,z_c_stresses,Location_c_stresses
0,0,10.000000,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,7.832483,225.948547,160.000000,-1923.530518,eta_y_bot_stresses
1,1,11.428571,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,7.879340,225.948547,154.285721,-1923.530518,eta_y_bot_stresses
2,2,12.857143,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,7.916116,225.948547,152.941177,-1923.530518,eta_y_bot_stresses
3,3,14.285714,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,7.951498,225.948547,152.941177,-1923.530518,eta_y_bot_stresses
4,4,15.714286,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,7.988680,225.948547,152.941177,-1923.530518,eta_y_bot_stresses
5,5,17.142857,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,8.025499,225.948547,152.941177,-1923.530518,eta_y_bot_stresses
6,6,18.571429,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,8.062981,225.948547,152.941177,-1923.530518,eta_y_bot_stresses
7,7,20.000000,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,8.101516,225.948547,152.941177,-1923.530518,eta_y_bot_stresses
8,8,21.428571,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,8.140179,225.948547,152.941177,-1923.530518,eta_y_bot_stresses
9,9,22.857143,5800,9720,1200,400,2780,90,0,0,...,534.6,0.0,-1658.7,bot,0,8.178907,225.948547,152.941177,-1923.530518,eta_y_bot_stresses


## Exploring an individual results dict


In [ ]:
# Input HERE
idx_s=1
id = 0
step='step_4'

In [ ]:
#load results dict
from verification import get_results
results=get_results(idx_s, id, step=None)

In [ ]:
results

In [ ]:
#print available keys of the dict
results[step]['GP'].keys()

In [ ]:
#specific discplacement
results[step]['nodal']['uz']['20']

In [ ]:
#specific stresses
results[step]['GP']['sig_sr_1L_x']['99']

In [ ]:
#calculating specific etas
from verification import *
concrete_bending_verification_stresses(structure=None, results = results, step=step, return_type='dict', verbalise=True)

## Check Valdiation functions

In [ ]:
# Input HERE
idx_s=1
id = 0
step='step_4'

In [ ]:
#load results dict
from verification import get_results
results=get_results(idx_s, id, step)

#### Check reinforcement steel bending verification

In [ ]:
#calculate maximum stresses
max_sig=[max(results['GP']['sig_sr_1L_x'].values()),
         max(results['GP']['sig_sr_1L_y'].values()),
         max(results['GP']['sig_sr_2L_x'].values()),
         max(results['GP']['sig_sr_2L_y'].values()),
         max(results['GP']['sig_sr_3L_x'].values()),
         max(results['GP']['sig_sr_3L_y'].values()),
         max(results['GP']['sig_sr_4L_x'].values()),
         max(results['GP']['sig_sr_4L_y'].values()),
]
print(max_sig)

In [ ]:
#!!compare with calculated eta_min_s
720/max(max_sig)

#### Check concrete bending verification (strains)

In [ ]:
#calculate minimum strains
min_eps=[min(results['GP']['eps_1_bot'].values()),
         min(results['GP']['eps_3_bot'].values()),
         min(results['GP']['eps_1_top'].values()),
         min(results['GP']['eps_3_top'].values())        
]
print(min_eps)

In [ ]:
#!!compare with calculated eta_min_c
-0.002/min(min_eps)

#### Check concrete bending verification (stresses)

In [ ]:
#calculate maximum negative stresses (--> minimum stresses)
# min_eps=[min(results['GP']['eps_1_bot'].values()),
#          min(results['GP']['eps_3_bot'].values()),
#          min(results['GP']['eps_1_top'].values()),
#          min(results['GP']['eps_3_top'].values())        
# ]
# print(min_eps)

In [ ]:
#!!compare with calculated eta_min_c_stresses
# fcc_eff/sig_c
# -0.002/min(min_eps)

In [ ]:
# results['GP']['fcc_eff_top'] 

In [ ]:
# results['GP']['sig_x_top']

## Plotting Runtimes

In [ ]:
#saving runtimes
n_elements1=[2450.0, 1601.0, 1142.0, 947.0,  947.0, 947.0, 947.0, 947.0, 947.0, 947.0]
analysis_time1=[1052.815,1011.080,649.90,633.543,632.563,637.594,641.200, 650.680, 650.564, 662.114] #seconds
analysis_time1 = [element / 60 for element in analysis_time1] #minutes

n_elements2= [362.0, 257.0,  206.0, 128.0, 128.0, 128.0,  128.0, 128.0,  128.0, 128.]
analysis_time2=[83.119,92.788 ,102.771,84.002,88.768 ,82.442,85.991,87.115, 84.954, 84.954]
analysis_time2 = [element / 60 for element in analysis_time2] #minutes

n_elements3=[2756.0, 1835.0, 1232.0, 1028.0, 1028.0, 1028.0, 1028.0, 1028.0, 1028.0, 1028.0]
analysis_time3=[1107.244,911.303, 570.685, 602.179, 594.138, 591.931, 593.499, 606.966, 603.610, 600.916]
analysis_time3 = [element / 60 for element in analysis_time3] #minutes


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(n_elements1, analysis_time1,label='L5800_parameterstudie (ms=1t)', marker='o', color='darkblue')
ax.plot(n_elements2, analysis_time2,label='L5800_parameterstudie (ms=3t)', marker='X', color='blue')
ax.plot(n_elements3, analysis_time3,label='L7000_parameterstudie (ms=1t)', marker='.', color='lightblue')


ax.set(xlabel='number of elements', ylabel='analysis time [minutes]',
       title='Runtime comparison')

# ax.set_ylim(bottom=0,top=20)

ax.grid()
ax.legend()

#fig.savefig("test.png")
plt.show()